In [1]:
## importing libraries
import numpy as np
import pandas as pd
import sklearn
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Importing Data
train = pd.read_csv('gcTrianingSet.csv')
test = pd.read_csv('gcPredictionFile.csv')

# Data Exploration

In [38]:
train.columns

Index(['initialUsedMemory', 'initialFreeMemory', 'query token', 'gcRun',
       'gcInitialMemory', 'gcFinalMemory', 'gcTotalMemory', 'userTime',
       'sysTime', 'realTime', 'cpuTimeTaken', 'finalUsedMemory',
       'finalFreeMemory', 'Memory'],
      dtype='object')

In [4]:
train.head()

,initialUsedMemory,initialFreeMemory,query token,gcRun,gcInitialMemory,gcFinalMemory,gcTotalMemory,userTime,sysTime,realTime,cpuTimeTaken,finalUsedMemory,finalFreeMemory
0,4.249634,2.999878,token_1,False,0.0,0.0,0.0,0.0,0.0,0.0,0.371230,4.409714,2.839798
1,4.409720,2.839792,token_2,False,0.0,0.0,0.0,0.0,0.0,0.0,0.220883,4.482361,2.767151
2,4.482361,2.767151,token_3,False,0.0,0.0,0.0,0.0,0.0,0.0,0.141776,4.542626,2.706886
3,4.542626,2.706886,token_4,False,0.0,0.0,0.0,0.0,0.0,0.0,0.156459,4.616293,2.633218
4,4.616296,2.633216,token_5,False,0.0,0.0,0.0,0.0,0.0,0.0,0.285218,4.787172,2.462339


In [8]:
np.setdiff1d(train['query token'],test['query token'])

array([], dtype=object)

therefore all tokens are present in train and test file.

In [11]:
train['gcRun'].value_counts()/train.shape[0]

False    0.937363
True     0.062637
Name: gcRun, dtype: float64

In [21]:
len(np.unique(train['query token']))

91

In [31]:
#train['Memory'] = train['finalUsedMemory'] - train['initialUsedMemory']

In [51]:
train.groupby('gcRun')['cpuTimeTaken'].describe()

,count,mean,std,min,25%,50%,75%,max
gcRun,,,,,,,,
False,2559.0,0.178492,0.116777,0.001052,0.111673,0.139285,0.225554,0.906536
True,171.0,0.307444,0.215037,0.082834,0.132598,0.234050,0.400445,0.873825


# Modelling

In [222]:
## modelling

training = train[['query token', 'cpuTimeTaken','initialFreeMemory']]
testing = test[['query token','cpuTimeTaken']]

In [223]:
# label encoding query token
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [224]:
combi = pd.concat([training,testing])
combi['query token'] = number.fit_transform(combi['query token'])

combi['query token'] = combi['query token'].astype('object')

training = combi[:training.shape[0]]
testing = combi[training.shape[0]:]

## Random forest

In [225]:
testing.drop('initialFreeMemory',axis=1,inplace=True)

## random forest

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=200, max_depth=3)

model.fit(training.drop('initialFreeMemory',axis=1), training['initialFreeMemory'])

pred = model.predict(testing)

/Users/shubhamjain/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [226]:
pred[pred > 0 ].shape

(1625,)

In [227]:
pred

array([ 1.98597474,  1.89028157,  1.93596205, ...,  1.81073608,
        1.88958885,  2.00862874])

In [230]:
test['initialFreeMemory'] = pred

test['gcRun'] = False

for i in range(0,test.shape[0]-1):
    if (test.loc[i+1, 'initialFreeMemory'] > test.loc[i,'initialFreeMemory']):
        test.loc[i,'gcRun'] = True
        
sub = test[['initialFreeMemory','gcRun']].reset_index()
sub.columns = ['serialNum','initialFreeMemory','gcRun']
sub['serialNum'] = sub['serialNum'] + 1

## for negative value, if any

sub.loc[sub['initialFreeMemory'] < 0,'initialFreeMemory'] = sub.loc[sub['initialFreeMemory'] < 0,'initialFreeMemory'] * -1


# Cross validation

In [213]:
training1 = training.groupby('query token').head(20)
cv = training.groupby('query token').tail(10)

In [220]:
## training on training1 dataset and predicting on cv to tune the parameters
from sklearn.metrics import mean_squared_error
for n in [100,200,400]:
    for d in [3,5,7,9]:
        model = RandomForestRegressor(n_estimators= n, max_depth=d, oob_score=True)
        model.fit(training1.drop('initialFreeMemory',axis=1), training1['initialFreeMemory'])
        
        pred = model.predict(cv.drop('initialFreeMemory',axis=1))
        
        print (n,d, mean_squared_error(cv['initialFreeMemory'], pred))

100 3 0.705847184338
100 5 0.704794058373
100 7 0.707573861093
100 9 0.721896440982
200 3 0.70342788431
200 5 0.706317878694
200 7 0.713802403784
200 9 0.723453186123
400 3 0.705470228966
400 5 0.707704544865
400 7 0.710875404333
400 9 0.724168666875


In [ ]:
## submission
sub.to_csv('Pred_RF.csv',index=False)

### Obtained a score of 302 on the leaderboard.

## XGBoost

In [235]:
## using xgboost model
import xgboost as xgb

xgb_para = {
    'eta':0.05,
    'max_depth':6,
    'subsample':0.75,
    'colsample_bytree':0.8,
    'min_child_weight':1,
    'objective':'reg:linear',
    'eval_metric':'rmse',

}

/Users/shubhamjain/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [240]:
#Dmatrix

training['query token'] = training['query token'].astype('int')
testing['query token'] = testing['query token'].astype('int')


dtrain = xgb.DMatrix(training.drop('initialFreeMemory',axis=1), training['initialFreeMemory'])
dtest = xgb.DMatrix(testing)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x11f7c80b8>>
Traceback (most recent call last):
  File "/Users/shubhamjain/anaconda/lib/python3.6/site-packages/xgboost/core.py", line 324, in __del__
    _check_call(_LIB.XGDMatrixFree(self.handle))
AttributeError: 'DMatrix' object has no attribute 'handle'
/Users/shubhamjain/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/shubhamjain/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [241]:
#xgboost cv
cv_result = xgb.cv(xgb_para,
                   dtrain,
                   num_boost_round=200,
                   early_stopping_rounds=50,
                   verbose_eval=50,
                   show_stdv=False)
num_boost_rounds = len(cv_result)
print (num_boost_rounds)

model = xgb.train(dict(xgb_para), dtrain,num_boost_round=num_boost_rounds)

[0]	train-rmse:1.53325	test-rmse:1.53348
[50]	train-rmse:0.703761	test-rmse:0.742537
[100]	train-rmse:0.675392	test-rmse:0.742814
74


In [244]:
pred = model.predict(dtest)

In [245]:
test['initialFreeMemory'] = pred

test['gcRun'] = False

for i in range(0,test.shape[0]-1):
    if (test.loc[i+1, 'initialFreeMemory'] > test.loc[i,'initialFreeMemory']):
        test.loc[i,'gcRun'] = True
        
sub = test[['initialFreeMemory','gcRun']].reset_index()
sub.columns = ['serialNum','initialFreeMemory','gcRun']
sub['serialNum'] = sub['serialNum'] + 1

## for negative value, if any

sub.loc[sub['initialFreeMemory'] < 0,'initialFreeMemory'] = sub.loc[sub['initialFreeMemory'] < 0,'initialFreeMemory'] * -1


In [ ]:
## submission
sub.to_csv('Pred_XGB.csv',index=False)

### Obtained a score of 314 on the leaderboard.

### Finished the competition with 47th Rank among 350 teams from top 25 engineering colleges.